# Rapidly-exploring Random Trees (RRT)
### Input: start: Tuple, goal: Tuple, obstacles: List[List[Tuple]], space_size: Tuple
// rrt(start, goal, obstacles, space_size), nearest_point(point, tree), colliding(p1, p2, obstacle),


In [1]:
def distance(p1, p2):
    return (p1[0]-p2[0])**2 + (p1[1]-p2[1])**2

In [2]:
import numpy

In [3]:
def nearest_point(point, tree):
    dist = [distance(point, x) for x in tree]
    res = tree[numpy.argmin(dist)]
    return res

In [48]:
nearest_point((0,0), [(1,1), (1,2), (2,2)])

(1, 1)

In [4]:
import pdb

In [15]:
def line_intersects(l1, l2):
    #pdb.set_trace()
    if (l1[0][0]<l2[0][0] and l1[1][0]>l2[0][0]) or (l1[0][0]<l2[1][0] and l1[1][0]>l2[1][0])\
        or l1[0][0]==l2[0][0] or l1[1][0]==l2[0][0] or l1[0][0]==l2[1][0] or l1[1][0]==l2[1][0]:
        if (l1[0][1]>l2[0][1] and l1[0][1]<l2[1][1]) or l1[0][1]==l2[0][1] or l1[0][1]==l2[1][1]:
            return True
        if (l1[1][1]>l2[0][1] and l1[1][1]<l2[1][1]) or l1[1][1]==l2[0][1] or l1[1][1]==l2[1][1]:
            return True 
        if (l1[0][1]<l2[0][1] and l1[1][1]>l2[1][1]) or l1[0][1]==l2[0][1] or l1[1][1]==l2[1][1]:
            return True
        if (l1[0][1]>l2[1][1] and l1[1][1]<l2[0][1]) or l1[0][1]==l2[1][1] or l1[1][1]==l2[0][1]:
            return True
    return False

In [16]:
def colliding(p1, p2, obstacle):
    #obstacle_polygon = {}
    lines = []
    for point in obstacle:
        temp = obstacle.copy()
        temp.remove(point)
        np1 = nearest_point(point, temp)
        temp.remove(np1)
        np2 = nearest_point(point, temp)
        #obstacle_polygon[point] = [np1, np2]
        if (point, np1) not in lines and (np1, point) not in lines:
            lines.append((point, np1))
        if (point, np2) not in lines and (np2, point) not in lines:
            lines.append((point, np2))
    resp = [line_intersects((p1, p2), x) for x in lines]
    return lines, resp

In [17]:
obs = [(0,0),(0,2), (2,2), (2,0)]
colliding((0,0), (1,1), obs)

([((0, 0), (0, 2)), ((0, 0), (2, 0)), ((0, 2), (2, 2)), ((2, 2), (2, 0))],
 [True, True, False, False])

In [24]:
def fixed_increment(p1, p2, d, space_size):
    if p1[0] == p2[0]:
        return (p1[0], min(p2[1]+d, space_size[1]))
    if p1[1] == p2[1]:
        return (min(p2[0]+d, space_size[0]), p2[1])
    m = (p1[1]-p2[1])/(p1[0]-p2[0])
    c = p2[1] - m*p2[0]
    a1 = (1+m*m)
    b1 = 2*(m*c - m*p2[1] - p2[0])
    c1 = p2[0]**2 + p2[1]**2 +c**2 - d**2 -2*c
    under = b1**2 - 4*a1*c1
    if under < 0:
        return (p1[0], min(p2[1]+d, space_size[1]))
    x = (-b1 + numpy.sqrt(under))/(2*a1)
    y = m*x + c
    x = min(max(0,x), space_size[0])
    y = min(max(0,y), space_size[1])
    return (x, y)

In [25]:
fixed_increment((0,0), (2,2), 0.2, (5,5))

(2.1414213562373097, 2.1414213562373097)

In [8]:
def goal_isvalid(goal, obstacles):
    for obstacle in obstacles:
        minx = min(obstacle, key=lambda x: x[0])[0]
        miny = min(obstacle, key=lambda x: x[1])[1]
        maxx = max(obstacle, key=lambda x: x[0])[0]
        maxy = max(obstacle, key=lambda x: x[1])[1]
        if goal[0]>= minx and goal[0]<=maxx\
        and goal[1]>=miny and goal[1]<=maxy:
            return False
    return True

In [44]:
goal_isvalid((1,1), [[(0,0), (0,2), (2,2), (2,0)]])

False

In [27]:
def rrt(start, goal, obstacles, space_size, d=0.1):
    tree = [start]
    connections = []
    if not goal_isvalid:
        return "Path not possible"
    goal_reachable = not any([any(colliding(start, goal, obstacle)[1]) for obstacle in obstacles])
    #print(goal_reachable)
    while not goal_reachable:
        new_point = tuple(numpy.random.uniform(space_size))
        nearest = nearest_point(new_point, tree)
        new_point = fixed_increment(new_point, nearest, d, space_size=space_size)
        if not any([any(colliding(nearest, new_point, obstacle)[1]) for obstacle in obstacles]):
            tree.append(new_point)
            connections.append((nearest, new_point))
            goal_reachable = not any([any(colliding(new_point, goal, obstacle)[1]) for obstacle in obstacles])
    return tree, connections

In [41]:
min([(1,0), (2, 0), (3,4), (0,5)], key=lambda x: x[0])

(0, 5)

In [29]:
rrt((0,0), (5,5), [[(0,1), (1,1), (1,2), (0,2)]], space_size=(6,6))

([(0, 0),
  (0.027883517380099216, 0.09603389744519227),
  (0.29660907156423694, 0.34894575869205097),
  (3.9119321307470267, 0.448945758692051)],
 [((0, 0), (0.027883517380099216, 0.09603389744519227)),
  ((0.027883517380099216, 0.09603389744519227),
   (0.29660907156423694, 0.34894575869205097)),
  ((0.29660907156423694, 0.34894575869205097),
   (3.9119321307470267, 0.448945758692051))])

In [18]:
colliding((0,0), (5,5), [(0,1), (1,1), (1,2), (0,2)])

([((0, 1), (1, 1)), ((0, 1), (0, 2)), ((1, 1), (1, 2)), ((1, 2), (0, 2))],
 [True, True, True, True])